In [71]:
from collections import defaultdict
import numpy as np
import random
import itertools
import re
import math

trainingSetTextFile = "dataset/training_set_clean_only_text.txt"
trainingSetTagsFile = "dataset/training_set_clean_only_tags.txt"

In [112]:
testData = []
tagsData = []
data = []

with open(trainingSetTextFile, mode='r', encoding='utf8') as f:
    # here we load learning data
    n = 1000000
    while n > 0:
        c = f.readline()
        if not c:
            print("End of file")
            break

        #c = c.lower()
        #c = re.split(SPLIT_STRING, c)
        #c = c.split()   
        #c = splitWords(c)
        
        testData.append(c)
        
        n -= 1
        
with open(trainingSetTagsFile, mode='r', encoding='utf8') as f:
    # here we load learning data
    n = 1000000
    while n > 0:
        c = f.readline()
        if not c:
            print("End of file")
            break

        #c = c.lower()
        #c = re.split(SPLIT_STRING, c)
        #c = c.split()   
        #c = splitWords(c)
        
        tagsData.append(c)
        
        n -= 1
        
for i in range(0, len(testData)):
    data.append((testData[i], int(tagsData[i])))

End of file
End of file


In [114]:
print(data)

[('Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.\n', 0), ('@anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe\n', 0), ('@anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.\n', 0), ('@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.\n', 0), ('Odrzut natychmiastowy, kwaśna mina, mam problem\n', 0), ('Jaki on był fajny xdd pamiętam, że spóźniłam się na jego pierwsze zajęcia i to sporo i za karę kazał mi usiąść w pierwszej ławce XD\n', 0), ('@anonymized_account No nie ma u nas szczęścia 😉\n', 0), ('@anonymized_account Dawno kogoś tak wrednego nie widziałam xd\n', 0), ('@anonymized_account @anonymized_account Zaległości były, ale ważne czy były wezwania do zapłaty z których się klub nie wywiązał.\n', 0), ('@anonymized_account @anonymized_account @anonymized_account Gdzie jest @anonymized_account . Brudziński jesteś kłamcą i marnym kutas

In [74]:
def split(sentence):
    words = sentence.split()
    return words

### Naive Bayes Classifier

In [117]:
wordClassifier = defaultdict(dict)

# preprocessing for each word
for i in range(0, len(data)):
    (sentence, res) = data[i]
    res = int(res)
    for word in split(sentence):
        if res == 0:
            if word in wordClassifier:
                wordClassifier[word][0] += 1
            else:
                wordClassifier[word] = [1, 0]
        else:
            if word in wordClassifier:
                wordClassifier[word][1] += 1
            else:
                wordClassifier[word] = [0, 1]
                
print(wordClassifier)
#print(wordClassifier["kutasem"])

defaultdict(<class 'dict'>, {'Dla': [25, 1], 'mnie': [231, 7], 'faworytem': [3, 0], 'do': [841, 67], 'tytułu': [2, 0], 'będzie': [267, 16], 'Cracovia.': [3, 0], 'Zobaczymy,': [2, 0], 'czy': [335, 22], 'typ': [1, 0], 'się': [1573, 126], 'sprawdzi.': [1, 0], '@anonymized_account': [12319, 1413], 'Brawo': [16, 0], 'ty': [79, 54], 'Daria': [4, 0], 'kibic': [4, 0], 'ma': [460, 40], 'być': [176, 17], 'na': [1774, 139], 'dobre': [23, 5], 'i': [1639, 176], 'złe': [6, 0], 'Super,': [2, 0], 'polski': [7, 1], 'premier': [3, 0], 'składa': [1, 0], 'kwiaty': [5, 0], 'grobach': [1, 0], 'kolaborantów.': [1, 0], 'Ale': [217, 14], 'doczekaliśmy': [1, 0], 'czasów.': [1, 0], 'Musi.': [1, 0], 'Innej': [2, 0], 'drogi': [3, 0], 'nie': [2354, 224], 'mamy.': [2, 0], 'Odrzut': [1, 0], 'natychmiastowy,': [1, 0], 'kwaśna': [1, 0], 'mina,': [1, 0], 'mam': [204, 1], 'problem': [25, 1], 'Jaki': [22, 4], 'on': [63, 7], 'był': [155, 14], 'fajny': [12, 0], 'xdd': [5, 0], 'pamiętam,': [3, 0], 'że': [1193, 89], 'spóźniła

In [120]:
countNeutral = 0
countHate = 0
pC = 0

for i in range(0, len(data)):
    (fst, res) = data[i]
    res = int(res)
    if res == 0:
        countNeutral += 1
    else:
        countHate += 1
        
pC = countNeutral / (countNeutral + countHate)
print(countNeutral)
print(countHate)
print(pC)

# we should now count all p(W1|C)p(W2|C)...p(Wn|C)

9190
851
0.915247485310228


In [121]:
def naivebayes(sentence):
    words = split(sentence)
    
    # we count log (P(C)*P(C|W1)*P(C|W2)*...*P(C|Wn)) = log (P(C)) + log P(C|W1) + log P(C|W2) + ... + log P(C|Wn)
    logSum = math.log(pC)
    for word in words:
        (neutral, hate) = wordClassifier[word]
        if (neutral + hate != 0): # means we do have that in dictionary
            neutralProbability = (neutral) / (neutral + hate)
            if (neutralProbability == 0):
                neutralProbability = 0.000000001
            logSum += math.log(neutralProbability)
            #print(neutral)
            #print(hate)
            #print(neutralProbability)
            # find p(W1|C)
        else:
            print("ERROR, WORD NOT FOUND!!!!")
    
    prob = math.exp(logSum)
    return prob

In [110]:
test = "To jest test"
res = naivebayes(test)
print("Probability that: '" + test + "' is neutral: " + str(res))

Probability that: 'To jest test' is neutral: 0.07053977696211446


In [122]:
result = []
for i in range(0, len(data)):
    (sentence, res) = data[i]
    prob = naivebayes(sentence)
    result.append(prob)
    
# we will shrink the result into 0, 1 classifier
classifier = []
for i in range(0, len(result)):
    if (result[i] >= 0.5):
        classifier.append(1)
    else:
        classifier.append(0)
        
classifier = result
        
for i in range(0, len(data)):
    (sent, tag) = data[i]
    print(sent + "|" + str(tag) + "|" + str(classifier[i]))

Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.
|0|0.6484658884699639
@anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe
|0|0.2524941241846465
@anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.
|0|0.5614567632820802
@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.
|0|0.6725822639306743
Odrzut natychmiastowy, kwaśna mina, mam problem
|0|0.8757527532987173
Jaki on był fajny xdd pamiętam, że spóźniłam się na jego pierwsze zajęcia i to sporo i za karę kazał mi usiąść w pierwszej ławce XD
|0|0.22395920151717416
@anonymized_account No nie ma u nas szczęścia 😉
|0|0.573957888141138
@anonymized_account Dawno kogoś tak wrednego nie widziałam xd
|0|0.6303183846024543
@anonymized_account @anonymized_account Zaległości były, ale ważne czy były wezwania do zapłaty z których się klub nie wywiązał.
|0|0.4015371045895198
@anonymized_account @anonym

Gdzie jest ten suchar @anonymized_account ?
|0|0.5525699349860679
@anonymized_account I niezależnie od tego czy ktoś ma tytuł naukowy czy nie. Zauważyłem nawet, że bardziej u osób utytułowanych
|0|0.36120877091270054
@anonymized_account Yerri Mina pewnie wskoczy za tydzień.
|0|0.726178830287041
Glik: - Jestem od tygodnia w treningu. Byłem do dyspozycji trenera i gotowy, żeby zagrać.
|0|0.5131736555461779
RT @anonymized_account Glik: - Jestem od tygodnia w treningu. Byłem do dyspozycji trenera i gotowy, żeby zagrać.
|0|0.3946019472352398
@anonymized_account @anonymized_account pewnie robi sobie darmową reklame.Chce żeby o nim pisano.
|0|0.45742052525882027
@anonymized_account Pani Kamila podobno jest prawnikiem..❕❕❔❓👎👇👎
|0|0.19796406449783327
RT @anonymized_account @anonymized_account Pani Kamila podobno jest prawnikiem..❕❕❔❓👎👇👎
|0|0.15222333510146108
@anonymized_account Z sercem na jednej dłoni i kutasem w drugiej.
|1|1.248659093626792e-19
Czuje się jak zmoknięta kura przejachana przez

|0|0.23442767341464862
Cute muszę szybko iść do apteki po żelazo żeby nie umrzeć bo wczoraj kompletnie zapomniałam
|0|0.20191822185797922
@anonymized_account Dwa razy w życiu ja widziałam xdddd
|0|0.5225201161879298
@anonymized_account Wiadomo ze z dziećmi wyjście na Gubałówke to większe wyzwanie jak na rysy bez dzieci ;)
|0|0.4011388675403358
@anonymized_account @anonymized_account Kupcieu jeszcze sukienkę i postawcie na kratce wentylacyjnej metra.
|0|0.5647000716440004
@anonymized_account Powiedziałem do domu i do roboty!
|0|0.556558572639764
@anonymized_account :-) Nie ma też niemobilnego wózka. Chyba, że zepsuty.
|0|0.6336695107240222
@anonymized_account @anonymized_account ale czy CHODZILIŚCIE a nie czy CHODZICIE
|0|0.5200201580038277
@anonymized_account @anonymized_account Rozpieszczeni czym ? Wpierdolem od Macedończyków ?
|0|0.5709104274731187
@anonymized_account @anonymized_account @anonymized_account Chętnie bym coś zajebał albo dał lapóweczkę, ale nie mogę się przełamać, jest

@anonymized_account nie tylko Ty :) witaj w klubie :)
|0|0.4035795243170522
@anonymized_account Skąd się tacy ludzie biorą...
|0|0.5420149851999643
Coś mi miga przed oczami
|0|0.5769998290903033
@anonymized_account Wisła ma już macki w całej Polsce :)
|0|0.5730243229731042
@anonymized_account @anonymized_account 30 tyś na kampanię, taki browarnik wielki a tak ściubi, macie to w genach wy polscy BYZNESMENI
|0|0.12305182964400274
@anonymized_account W sensie ta kobieta co wstawiła to zdjęcie nie
|0|0.5956717336009257
@anonymized_account Bardzo słusznie. Nigdy więcej PiSu.
|0|0.14704836219549142
jeszcze raz zobacze na playerze reklame koncertu niki to rzuce komputerem i promise
|0|0.623315283300627
@anonymized_account @anonymized_account @anonymized_account @anonymized_account a w finale PP to nie zmarnowaliście szansy ? :P
|0|0.41047200883158064
@anonymized_account Ale chyba w hokeja 😀
|0|0.6454896472324202
@anonymized_account wiadomo co z Magdą ? @anonymized_account
|0|0.561598871226213

|0|0.4428512847699099
@anonymized_account @anonymized_account @anonymized_account @anonymized_account @anonymized_account @anonymized_account Mam propozycję: zmieńcie dealera bo zbyt łatwo odlatujecie.
|0|0.1189293692871451
@anonymized_account Albo: \n- Jaki tytuł miała ta piosenka?\n- Która?\n- No ta \"na na na naa naaa na.
|0|0.48854606056368594
@anonymized_account @anonymized_account No pewnie, \"must have\" na insta  😁
|0|0.6522688000114699
@anonymized_account Jesli oni go ściągnęli za 21 baniek, sprzedając Paulinho za 50, to świat stanął na głowie 😀
|0|0.38077052321136867
@anonymized_account @anonymized_account Narazie var pomaga tylko legi czy to normalne??
|0|0.5818241794460498
@anonymized_account @anonymized_account Musimy poczekać na Młodego, bo ja mogę ewentualnie odpisać, że czarny 😬
|0|0.4660189600768756
@anonymized_account Zdjęcie ze Shetyną :) każdy kto ma taką fotkę źle kończy :) ostatnio pecha miał byly już ambasador USA zdaje się 😂😂😂😂 ?
|0|0.43668075941081097
RT @anony

...

RT @anonymized_account Słów kilka o żałosnym antybohaterze ostatnich dni. \nhttps://t.co/Rzlzidb6l0
|0|0.6525130634951468
@anonymized_account @anonymized_account Oj, a czemu on w ortopedycznych butach musi chodzić?
|0|0.34468894202512673
@anonymized_account @anonymized_account Nie interesują mnie Bałkany złodzieju publicznych pieniędzy.
|0|0.3383440567830693
@anonymized_account @anonymized_account Po prostu normalnie spierdoliłeś z tamtąd i nie dorabiają do tego teorii.
|0|0.34293216934141724
@anonymized_account @anonymized_account Przecież nie od dzisiaj wiadomo że polscy kierowcy to albo pijaki albo debile.
|1|2.63673516248749e-28
@anonymized_account tworzą precedens,  chcą iść na wojnę, rękawica została rzucona
|0|0.6454143523770555
Pazdan chyba zapomnial, że są kamery na stadionie. Klasy się nie kupi.
|0|0.5689937664242589
@anonymized_account Chyba jest to dopinane jeszcze.
|0|0.6513503458644996
@anonymized_account @anonymized_account @anonymized_account Zarząd ma na celu dobro klu

|0|0.3179531192603053
@anonymized_account @anonymized_account mowa o tej dupie ?:)
|1|0.31021968602974137
@anonymized_account Ja chcę na Ślunsku Hardcore Core, nie byłem
|0|0.6599004296130154
@anonymized_account @anonymized_account @anonymized_account @anonymized_account słowo \"gości\" zostało wyciszone 😜
|0|0.5532761668939317
Od paliwa też za PO, za pis weszło w życie, te szczury kłamią
|1|6.102685932048901e-29
@anonymized_account Albo po prostu sędzia jest dziś bardzo słaby.
|0|0.5620772350444482
@anonymized_account @anonymized_account lepiej napisać że dostali wpierdol.A nie, bo ludzie którzy dostali po ryju by się mogli głupio poczuć.
|1|8.510803023030749e-21
@anonymized_account to o tobie, nienawidzisz Polaków genetycznie
|1|5.419132401395883e-28
@anonymized_account Meza, Pavon, w eliminacjach był jeszcze gagatek Benedetto. Druga linia to Mascherano, Perez, Biglia. Ludzie...
|0|0.29042354201756865
Pestkę z tej brzoskwinki ja wyciagnę
|0|0.7130512862031989
@anonymized_account Legi

Losowanie się nie skończyło, ale i tak Ligę Mistrzów wygrał już ten, co zrobił ostatnią reformę rozgrywek.
|0|0.4911897560893926
@anonymized_account @anonymized_account @anonymized_account To czemu od razu po prostu nie kupiła biletu tylko skontaktowała się z klubem ?
|0|0.3056145530711321
.@anonymized_account Dwukrotnie zamówili szkła o złej wadzie, raz uszkodzili przy montażu. Czekam miesiąc. Zero poczucia winy. Odradzam.👎😡
|0|0.3188090214800105
@anonymized_account Żeby tylko na legie te dziewczyny niewyszly zamiast naszych grajkow 😊
|0|0.42495254572331215
@anonymized_account @anonymized_account Spadniemy !! Przecież nie ma Wolskiego !! Tak mówią dzieci na fb i tt a ja im wierze !:) 1 liga wita :)
|0|0.0765440387057812
@anonymized_account @anonymized_account np Belgia, Serbia, Kazachstan.W Holandii i Grecji są jakieś dziwne playoffy o wejscie do europejskich pucharow
|0|0.46977644185477363
@anonymized_account I to do ntej potęgi!😁😁
|0|0.6032774138784883
@anonymized_account 100% konie

|0|0.5087469344759578
RT @anonymized_account Jest decyzja sądu ws. Stanisława Gawłowskiego. Polityk PO idzie do więzienia\n@anonymized_account #wieszwiecej\nhttps://t.co/evVvuT3qZO
|0|0.3911980453877902
asocjacja intymna władzy z księdzem Rydzykiem jest przerażająca
|0|0.02529808620960049
Czy będą nagrody za oddanie nagrody?
|0|0.5200902492036732
RT @anonymized_account Czy będą nagrody za oddanie nagrody?
|0|0.3999204223673619
@anonymized_account Kolejny debilny sukience. Ich seksualne obsesje zaczynają przybierać absurdalne rozmiary
|1|7.894904218345055e-56
@anonymized_account @anonymized_account Normalny Polak za granicą złego słowa nie powie na swój kraj, sprzedajna świnia tak.
|1|9.635005835417974e-13
RT @anonymized_account @anonymized_account @anonymized_account Normalny Polak za granicą złego słowa nie powie na swój kraj, sprzedajna świnia tak.
|1|7.408782627846157e-13
@anonymized_account @anonymized_account @anonymized_account Tak jest. Ale Macierewicz jest jednym i drugim.
|0|0

@anonymized_account Z tego co mi wiadomo tam są dwie kolejki po bilet jedna a do wagonów druga. Jeśli się.zmienilo to sprostować proszę
|0|0.21656841356298834
@anonymized_account dłuuuuuuga droga przede mną XD
|0|0.7526475354822294
@anonymized_account Musiał jest na poziomie Gila, ale jest medialny i sympatyczny dlatego jest kreowany na jednego z najlepszych w Polsce.
|0|0.1904943306072289
@anonymized_account A ile lot będzie opóźniony?
|0|0.6290072504306602
@anonymized_account @anonymized_account skoro nie ma to musieli usunąć bo tam  wszystkie artykuły są tyle że niektóre zablokowane
|0|0.3766515317441768
@anonymized_account I nietrafiony karny przez Dinamo w 96 minucie :)
|0|0.5895863667630176
Nikt tak dobrze jak Donald Tusk nie podsumował rządów Po-PSL.Po raz pierwszy powiedział prawdę.
|0|0.16562473737871122
listopad jak LIŚCI OPAD\nmój mózg pracuje już na innych obrotach
|0|0.27150379190970525
RT @anonymized_account listopad jak LIŚCI OPAD\nmój mózg pracuje już na innych obrotach

## Training data distribution

Dividing data into segments so that we have both training data and test data

In [153]:
def distributeData(data, ratio):
    counter = ratio * len(data)
    trainingData = []
    testData = []
    for i in range(0, len(data)):
        if i <= counter:
            trainingData.append(data[i])
        else:
            testData.append(data[i])
    return (trainingData, testData)

def randDistributeData(data, ratio):
    random.shuffle(data)
    return distributeData(data, ratio)